In [4]:
!nvidia-smi

Wed Jul 19 00:55:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 516.40       Driver Version: 516.40       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   54C    P8     1W /  N/A |      0MiB /  6144MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
import torch
from torch import nn

In [6]:
torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [7]:
torch.cuda.device_count()

1

In [8]:
def try_gpu(i=0):
    if torch.cuda.device_count() >= i+1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

In [9]:
def try_all_gpus():
    devices = [torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

In [10]:
try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [11]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [12]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [13]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.2584, 0.9479, 0.3434],
        [0.7027, 0.7359, 0.5243]])

In [14]:
Z = X.cuda(1)  # 将 gpu：0 上的 X 复制到 gpu：1
print(X)
print(Z)

RuntimeError: CUDA error: invalid device ordinal
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
Y + Z

In [ ]:
Z.cuda(1) is Z

In [15]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [16]:
net(X)

tensor([[0.4429],
        [0.4429]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [17]:
net[0].weight.data.device

device(type='cuda', index=0)